In [1]:
import pandas as pd

df = pd.read_csv (r'~/Downloads/values.csv')

In [2]:
df.describe()

,a,b,c
count,2.930000e+02,293.000000,293.000000
mean,-3.793106e-06,0.097745,-532.208275
std,9.057253e-07,0.017059,87.844236
min,-1.191466e-05,0.062540,-1112.948629
25%,-4.243026e-06,0.086497,-582.197177
50%,-3.729551e-06,0.096546,-526.825773
75%,-3.183655e-06,0.107152,-475.494365
max,-1.884254e-06,0.237608,-363.625559


# Load Values From File

In [ ]:
import glob

path = r'../data_tue/*' # use your path
all_folders = glob.glob(path)


li = []
df_out=pd.DataFrame(columns=['id','real_temp','adc_value'])
# dl=pd.DataFrame(columns=['real_temp','header','frimware_adc','col4','col5','col6','col7','col8','col9','col10','col11','col12','col3','col4','col15'])
i=1
j=1
for folder in all_folders:
    df=pd.read_csv(folder+'/data.txt', index_col=None, header=None)
    for row in df.iterrows():
        if row[1][3]==265:
            df_out.loc[i]=folder.split('/')[2],row[1][0],row[1][5]
        else:
            df_out.loc[i]=folder.split('/')[2],row[1][0],row[1][6]
        i+=1
    if j%10 == 0:
        print ("Loading file number:........."+str(j))
    j+=1
    #     df = pd.read_csv(folder+'/data.txt', index_col=None, header=0)
# frame = pd.concat(li, axis=0, ignore_index=True)

Loading file number:.........10
Loading file number:.........20
Loading file number:.........30
Loading file number:.........40
Loading file number:.........50
Loading file number:.........60
Loading file number:.........70
Loading file number:.........80
Loading file number:.........90
Loading file number:.........100
Loading file number:.........110
Loading file number:.........120
Loading file number:.........130
Loading file number:.........140
Loading file number:.........150
Loading file number:.........160
Loading file number:.........170
Loading file number:.........180
Loading file number:.........190
Loading file number:.........200
Loading file number:.........210
Loading file number:.........220
Loading file number:.........230
Loading file number:.........240
Loading file number:.........250
Loading file number:.........260
Loading file number:.........270
Loading file number:.........280


In [ ]:
df_out.info()

In [ ]:
df_out.head()

In [12]:
df_out['adc_value'].describe()

count    24745.000000
mean      9632.081633
std       5441.267152
min        757.000000
25%       8080.000000
50%       8791.000000
75%       9997.000000
max      64742.000000
Name: adc_value, dtype: float64

In [16]:
folder.split('/')[2]

'DWACYC0NGZFQN4ZKYPM3JP3ZWR'

In [ ]:
df_2=